<a href="https://colab.research.google.com/github/Jenil02/CS6910-Assignment2/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [2]:
import os
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torchvision
import pathlib
from torch.autograd import Variable
import glob
from torch.optim import Adam
from torch.utils.data import random_split
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cuda


In [5]:
train_path = '/content/drive/MyDrive/inaturalist_12K/train'
test_path = '/content/drive/MyDrive/inaturalist_12K/val'

def load_data(data_augment):
  if data_augment == True:
    transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize((256,256)),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.RandomRotation(30)
    ])
  else:
    transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize((256,256)),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

  trainset = torchvision.datasets.ImageFolder(train_path, transform=transform)

  testset = torchvision.datasets.ImageFolder(test_path, transform=transform)

  return trainset, testset

In [6]:
len(load_data(True))


2

In [7]:
class ConvNet(nn.Module):

  def __init__(self, num_classes, num_filters, kernel_size, droprate, batch_norm, filter_org, activation_fn, data_augment):
    super(ConvNet,self).__init__()
      
    #Output size after convolution filter
    #((w-f+2P)/s) + 1

    # self.conv1 = nn.Conv3d(3, 12, kernel_size=3, stride=1, padding=1)
    #shape = (32, 12, 256, 256)
    self.num_filters = num_filters
    self.kernel_size = kernel_size
    self.num_classes = num_classes
    self.droprate = droprate
    self.batch_norm = batch_norm
    self.activation = activation_fn
    self.filter_org = filter_org
    # self.train_dir = train_dir
    # self.test_dir = test_dir
    self.data_augment = data_augment

    size = (32, 3, 256, 256)

    self.conv1 = nn.Conv2d(3, 3*num_filters, kernel_size, stride=1, padding=1)
    self.batch_norm1 = nn.BatchNorm2d(3*num_filters)
    self.pool = nn.MaxPool2d(3, stride=2)

    self.conv2 = nn.Conv2d(3*num_filters, 3*num_filters*(filter_org), kernel_size, stride=1, padding=1)
    self.batch_norm2 = nn.BatchNorm2d(3*num_filters*(filter_org))

    self.conv3 = nn.Conv2d(3*num_filters*(filter_org), 3*num_filters*(filter_org**2), kernel_size, stride=1, padding=1)
    self.batch_norm3 = nn.BatchNorm2d(3*num_filters*(filter_org**2))

    self.conv4 = nn.Conv2d(3*num_filters*(filter_org**2), 3*num_filters*(filter_org**3), kernel_size, stride=1, padding=1)
    self.batch_norm4 = nn.BatchNorm2d(3*num_filters*(filter_org**3))

    self.conv5 = nn.Conv2d(3*num_filters*(filter_org**3), 3*num_filters*(filter_org**4), kernel_size, stride=1, padding=1)
    self.batch_norm5 = nn.BatchNorm2d(3*num_filters*(filter_org**4))
    
    self.flatten = nn.Flatten(start_dim=1, end_dim = -1)

    for i in range(5):
      m = nn.Conv2d(3, 30, kernel_size, stride=1, padding=1)
      input = torch.randn(32, 3, size[2], size[3])
      output = m(input)
      n = nn.MaxPool2d(3, stride=2)
      output = n(output)
      size = output.shape
      input = output

    self.fc1 = nn.Linear(3*num_filters*(filter_org**4)*size[2]*size[3], num_classes)
    self.dropout = nn.Dropout(droprate)
    self.fc2 = nn.Linear(num_classes, num_classes)
    self.softmax = nn.Softmax(dim=1)

  def activation_fn(self,x):
    if self.activation == 'ReLU':
      return nn.functional.relu(x)
    if self.activation == 'GELU':
      return nn.functional.gelu(x)
    if self.activation == 'SiLU':
      return nn.functional.silu(x)
    if self.activation == 'Mish':
      return nn.functional.mish(x)
    if self.activation == 'LeakyReLU':
      return nn.functional.leaky_relu(x)

  def data_load(self):
    trainset, testset = load_data(self.data_augment)

    test_abs = int(len(trainset)*0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset)-test_abs])

    trainloader = DataLoader(train_subset, batch_size=32, shuffle=True)
    valloader = DataLoader(val_subset, batch_size=32, shuffle=True)
    return train_subset, val_subset, trainloader, valloader, testset
    

  def forward(self, inputs):
    self.inputs = inputs
    cur_in = inputs
    
    cur_out = self.conv1(cur_in)
    if self.batch_norm == True:
      cur_out = self.batch_norm1(cur_out)
    cur_out = self.activation_fn(cur_out)
    cur_out = self.pool(cur_out)

    cur_out = self.conv2(cur_out)
    if self.batch_norm == True:
      cur_out = self.batch_norm2(cur_out)
    cur_out = self.activation_fn(cur_out)
    cur_out = self.pool(cur_out)


    cur_out = self.conv3(cur_out)
    if self.batch_norm == True:
      cur_out = self.batch_norm3(cur_out)
    cur_out = self.activation_fn(cur_out)
    cur_out = self.pool(cur_out)
    
    cur_out = self.conv4(cur_out)
    if self.batch_norm == True:
      cur_out = self.batch_norm4(cur_out)
    cur_out = self.activation_fn(cur_out)
    cur_out = self.pool(cur_out)

    cur_out = self.conv5(cur_out)
    if self.batch_norm == True:
      cur_out = self.batch_norm5(cur_out)
    cur_out = self.activation_fn(cur_out)
    cur_out = self.pool(cur_out)

    cur_out = self.flatten(cur_out)

    cur_out = self.dropout(cur_out)

    cur_out = self.fc1(cur_out)

    cur_out = self.fc2(cur_out)

    cur_out = self.softmax(cur_out)
    return cur_out


In [11]:
def train_val(model, optimizer, num_epochs):

  train_subset, val_subset, trainloader, valloader, testset = model.data_load()

  loss_function = nn.CrossEntropyLoss()
  train_count = len(train_subset)
  val_count = len(val_subset)
  optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
  
  for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images,labels) in enumerate(trainloader):
      if torch.cuda.is_available():
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
      else:
        images = Variable(images)
        labels = Variable(labels)

      optimizer.zero_grad()

      outputs = model(images)
      loss = loss_function(outputs,labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.cpu().data*images.size(0)
      _,prediction = torch.max(outputs.data, 1)

      train_accuracy += int(torch.sum(prediction == labels.data))

    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count

    model.eval()

    val_loss = 0.0
    val_accuracy = 0.0

    for i, (images,labels) in enumerate(valloader):
      with torch.no_grad():
        if torch.cuda.is_available():
          images = Variable(images.cuda())
          labels = Variable(labels.cuda())
        else:
          images = Variable(images)
          labels = Variable(labels)

        outputs = model(images)
        loss = loss_function(outputs,labels)

        val_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data, 1)

        val_accuracy += int(torch.sum(prediction == labels.data))

    val_accuracy = val_accuracy/val_count
    val_loss = val_loss/val_count

    print({"train_loss": train_loss, "val_loss": val_loss, "train_accuracy": train_accuracy, "val_accuracy": val_accuracy})


In [ ]:
model = ConvNet(10, 3, 5, 0.1, True, 1, 'ReLU', True).to(device)
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
train_val(model, optimizer, 5)

In [ ]:
def run():
  wandb.init()
  config = wandb.config
  model = ConvNet(10, config.num_filters,  config.kernel_size, config.droprate, config.batch_norm, config.filter_org, config.activation, config.data_augment).to(device)
  optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
  print('hi')
  train_val(model, optimizer, 5)
  print('bye')
  wandb.finish()

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
        'goal' : 'maximize',
        'name': 'val_accuracy'
    },
    'parameters': {
        'num_filters': {
            'values': [16, 32, 64, 128]
        },
        'filter_org': {
            'values': [0.5, 1, 2, 3]
        },
        'kernel_size': {
            'values': [3, 5, 7]
        },
        'droprate': {
            'values': [0, 0.1, 0.2, 0.3]
        },
        'batch_norm': {
            'values': [True, False]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish', 'LeakyReLU']
        },
        'data_augment': {
            'values': [True, False]
        },
        
        }
    }

sweep_id = wandb.sweep(sweep_config, project="iNaturalist")
wandb.agent(sweep_id, run, count=2)